<a href="https://colab.research.google.com/github/Naveed101633/Advanced-RAG-Chatbot/blob/main/context_management_openai_agent_sdk_by_Naveed.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# *Install openai-agents SDK*

In [ ]:
!pip install -Uq openai-agents

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.9/121.9 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.3/129.3 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.2 MB/s eta 0:00:00


# *Agent Configuration*

In [ ]:
import nest_asyncio
nest_asyncio.apply()

In [ ]:
from pydantic import BaseModel
from agents import (
    AsyncOpenAI,
    OpenAIChatCompletionsModel,
    RunConfig
)

from google.colab import userdata
gemini_api_key = userdata.get('GOOGLE_API_KEY')

external_client = AsyncOpenAI(
    api_key= gemini_api_key,
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
)

model = OpenAIChatCompletionsModel(
  model= 'gemini-2.0-flash',
  openai_client= external_client
)

from agents import set_default_openai_client, set_tracing_disabled
set_tracing_disabled(True)
set_default_openai_client(external_client)

# *Local Context Use Case 1: Student Scenario*

In [ ]:
import asyncio
from dataclasses import dataclass
from agents import Agent, Runner, function_tool, RunContextWrapper
from openai.types.responses import ResponseTextDeltaEvent

@dataclass
class Student:
    name: str
    department: str
    roll_no: str
    cgpa: str

@function_tool
async def fetch_student_name(wrapper: RunContextWrapper[Student]) -> str:
    """Returns the student's name."""
    return wrapper.context.name

@function_tool
async def fetch_student_department(wrapper: RunContextWrapper[Student]) -> str:
    """Returns the student's department."""
    return f"{wrapper.context.name} is enrolled in {wrapper.context.department}."

@function_tool
async def fetch_student_cgpa(wrapper: RunContextWrapper[Student]) -> str:
    """Returns the student's CGPA as a string."""
    return f"{wrapper.context.cgpa}"

async def main():
    # Create student context
    student_info = Student(name="Naveed Ahmed", department="IT", roll_no=94, cgpa=3.45)

    # Create agent with type hint
    agent = Agent[Student](
        name="Assistant",
        instructions="You are a helpful assistant that provides student information using the provided tools.",
        tools=[fetch_student_name, fetch_student_department, fetch_student_cgpa],
        model=model  # Assumes model is defined (e.g., gemini-2.0-flash)
    )


    result = await Runner.run(
        starting_agent= agent,
        input= "What is the student's name and CGPA?",
        context= student_info
    )

    print(result.final_output)

# Run the program
if __name__ == "__main__":
    asyncio.run(main())

The student's name is Naveed Ahmed and their CGPA is 3.45.


# *Dynamic Instructioms with context*

# *Local Context Use Case 12: Poets*

In [ ]:
import asyncio
import random
from typing import Literal
from agents import Agent, Runner, RunContextWrapper

class Mehfil():
  def __init__(self, style: Literal["Latif","Jaun", "Iqbal"]):
    self.style = style


def Legend_Poets(
  wrapper: RunContextWrapper[Mehfil], agent: Agent[Mehfil]
) -> str:
  context = wrapper.context
  if context.style ==  "Latif":
    return "Respond like Shah Abdul Latif Bhittai"
  elif context.style == "Jaun":
    return "Respond like the poet Jaun Elia"
  elif context.style == "Iqbal":
    return "Respond like Allama Iqbal"
  else:
    return "Respond like a robot"


agent = Agent(
    name= "Poets",
    instructions= Legend_Poets,
    model= model
)


async def main():
  choice= random.choice(["Latif", "Jaun", "Iqbal"])
  context = Mehfil(style= choice)
  print(f"Using style {choice} \n")

  user_message = "Tell me Breakup poetry in roman urdu"
  print(f"User Message {user_message}")

  result = await Runner.run(agent, user_message, context= context)
  print(f"Assistant: {result.final_output}")



if __name__ == "__main__":
  asyncio.run(main())

Using style Jaun 

User Message Tell me Breakup poetry in roman urdu
Assistant: Aisay bichra hai woh, jaise koi khwab tha,
Aankh khuli toh bas, ek azaab tha.

Tanhaiyon ki ye nagri, ab mera sheher hai,
Dil toh jaise koi, toot'ta hua shajar hai.

Woh kehta tha sitare tod launga,
Magar dekho, aasman bhi toh be-asar hai.

Muhabbat ki kitaab toh, jal gayi hogi,
Kagaz hi kya bachega, dil bhi agar hai?

Ab Jaun, ye rona dhona chhod bhi de,
Woh shakhs toh kisi aur ka humsafar hai.



# New *Agent/LLM context*

In [ ]:
from agents import enable_verbose_stdout_logging
enable_verbose_stdout_logging

<function agents.enable_verbose_stdout_logging()>

In [ ]:
from agents import enable_verbose_stdout_logging
enable_verbose_stdout_logging
import asyncio
from dataclasses import dataclass
from agents import Agent, Runner, RunContextWrapper, function_tool

@dataclass
class UserInfo:
    name: str
    location: str

@function_tool
async def greet_user(wrapper: RunContextWrapper[UserInfo], greetings: str) -> str:
    """Greet user with their name.

    Args:
        greetings: A specialized greeting for user
    """
    name = wrapper.context.name
    return f"Hello {name}, {greetings}"

# Define agent at module level
agent = Agent[UserInfo](
    name="Assistant",
    tools=[greet_user],
    model=model,  # Assumes model is defined (e.g., gemini-2.0-flash)
    instructions="Always greet the user using the greet_user tool and welcome them to University of Sindh."
)

async def main():
    user_info = UserInfo(name="Soomro", location="Pakistan")

    result = await Runner.run(
        starting_agent=agent,
        input="Greet the student",  # Changed to trigger tool
        context=user_info
    )

    print(f"Assistant: {result.final_output}")

if __name__ == "__main__":
    asyncio.run(main())

Assistant: Hello Soomro, Welcome to University of Sindh!

